In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '947688', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '283412', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '775051', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",May-21,"25,580","24,993",587,2.3,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",May-21,"99,524","97,175","2,349",2.4,01003
38629,CN0100500000000,1,5,"Barbour County, AL",May-21,"8,103","7,732",371,4.6,01005
38630,CN0100700000000,1,7,"Bibb County, AL",May-21,"8,555","8,293",262,3.1,01007
38631,CN0100900000000,1,9,"Blount County, AL",May-21,"24,709","24,238",471,1.9,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",May-21,"13,220","11,808",1412,10.7,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",May-21,"2,637","2,316",321,12.2,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",May-21,"7,079","6,267",812,11.5,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",May-21,"8,471","7,606",865,10.2,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",May-21,"9,433","8,309",1124,11.9,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1615056,2021-08-13,Webster,West Virginia,54101.0,633,9.0
1615057,2021-08-13,Wetzel,West Virginia,54103.0,1522,32.0
1615058,2021-08-13,Wirt,West Virginia,54105.0,479,4.0
1615059,2021-08-13,Wood,West Virginia,54107.0,8285,177.0
1615060,2021-08-13,Wyoming,West Virginia,54109.0,2202,41.0
1615061,2021-08-13,Adams,Wisconsin,55001.0,2063,21.0
1615062,2021-08-13,Ashland,Wisconsin,55003.0,1434,15.0
1615063,2021-08-13,Barron,Wisconsin,55005.0,6185,86.0
1615064,2021-08-13,Bayfield,Wisconsin,55007.0,1323,21.0
1615065,2021-08-13,Brown,Wisconsin,55009.0,36742,310.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_PROBABLE_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_PROBABLE_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_PROBABLE_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_PROBABLE_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_PROBABLE_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
524,08/06/2021,1464,429,102,7,1,1297,1728,71,5,6,192,80,24,0,0,176,79,255,14,1,1,490,126,30,4,0,428,129,557,22,2,2,261,70,15,1,0,253,81,334,9,1,1,385,117,23,1,0,319,112,430,20,1,1,136,35,12,1,0,121,29,150,8,1,1,11000
525,08/07/2021,1080,351,70,10,2,1334,1768,72,6,7,165,61,18,2,0,182,78,260,15,1,1,351,114,18,4,0,442,133,575,21,2,2,167,49,10,1,0,256,80,336,9,1,1,278,94,19,2,0,329,111,440,20,2,2,119,32,5,1,1,125,32,157,8,1,1,11000
526,08/08/2021,1008,316,68,6,2,1361,1792,76,6,7,142,49,18,1,0,189,76,265,17,1,1,305,94,17,2,0,444,134,578,22,2,2,149,59,10,0,0,257,79,336,10,1,1,285,90,16,2,0,338,110,448,20,1,1,127,22,8,1,0,133,31,164,8,1,1,11000
527,08/09/2021,1655,465,83,4,5,1382,1814,79,6,8,235,93,18,0,0,191,77,268,18,1,1,574,119,34,1,0,455,132,587,24,2,2,270,75,12,2,0,254,76,330,11,1,1,416,137,19,1,0,344,114,458,20,1,1,159,35,1,0,0,138,32,170,8,1,1,11000
528,08/10/2021,1318,455,32,6,1,1356,1785,72,6,8,210,89,3,0,0,191,77,268,16,1,1,440,142,12,4,0,449,135,584,23,3,3,243,73,5,1,0,243,71,314,10,1,1,297,123,7,0,0,338,113,451,17,1,1,128,27,5,1,0,135,31,166,7,1,1,11000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
524,08/06/2021,192,0,New York,Bronx,36005.0
525,08/07/2021,165,2,New York,Bronx,36005.0
526,08/08/2021,142,1,New York,Bronx,36005.0
527,08/09/2021,235,0,New York,Bronx,36005.0
528,08/10/2021,210,0,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
524,08/06/2021,490,4,Kings,New York,36047.0
525,08/07/2021,351,4,Kings,New York,36047.0
526,08/08/2021,305,2,Kings,New York,36047.0
527,08/09/2021,574,1,Kings,New York,36047.0
528,08/10/2021,440,4,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,5089,31


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
515,07/28/2021,281,1,New York,Queens,36081.0,2021-07-28
516,07/29/2021,311,4,New York,Queens,36081.0,2021-07-29
517,07/30/2021,315,1,New York,Queens,36081.0,2021-07-30
518,07/31/2021,208,0,New York,Queens,36081.0,2021-07-31
519,08/01/2021,216,4,New York,Queens,36081.0,2021-08-01
520,08/02/2021,375,1,New York,Queens,36081.0,2021-08-02
521,08/03/2021,343,0,New York,Queens,36081.0,2021-08-03
522,08/04/2021,359,1,New York,Queens,36081.0,2021-08-04
523,08/05/2021,344,2,New York,Queens,36081.0,2021-08-05
524,08/06/2021,385,1,New York,Queens,36081.0,2021-08-06


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
479,06/22/2021,Richmond,New York,36085.0,17,0.0,2021-06-22
480,06/23/2021,Richmond,New York,36085.0,23,0.0,2021-06-23
481,06/24/2021,Richmond,New York,36085.0,29,0.0,2021-06-24
482,06/25/2021,Richmond,New York,36085.0,23,1.0,2021-06-25
483,06/26/2021,Richmond,New York,36085.0,24,0.0,2021-06-26
484,06/27/2021,Richmond,New York,36085.0,22,0.0,2021-06-27
485,06/28/2021,Richmond,New York,36085.0,30,2.0,2021-06-28
486,06/29/2021,Richmond,New York,36085.0,34,0.0,2021-06-29
487,06/30/2021,Richmond,New York,36085.0,27,0.0,2021-06-30
488,07/01/2021,Richmond,New York,36085.0,33,0.0,2021-07-01


In [46]:
print (data_date_dt)

2021-08-13 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1608662,1001.0,114.0
1608663,1003.0,342.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,8715


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1614911,51690.0,79.0
1614912,51115.0,14.0
1614913,51117.0,68.0
1614914,51119.0,22.0
1614915,51121.0,81.0
1614916,51125.0,13.0
1614917,51127.0,19.0
1614918,51700.0,244.0
1614919,51710.0,278.0
1614920,51131.0,37.0


In [50]:
covid_death.shape

(6499, 2)

In [51]:
print (County_covid_death)

{1001.0: 114.0, 1003.0: 344.0, 1005.0: 65.0, 1007.0: 67.0, 1009.0: 140.0, 1011.0: 41.0, 1013.0: 72.0, 1015.0: 339.0, 1017.0: 125.0, 1019.0: 49.0, 1021.0: 117.0, 1023.0: 25.0, 1025.0: 63.0, 1027.0: 59.0, 1029.0: 45.0, 1031.0: 137.0, 1033.0: 143.0, 1035.0: 33.0, 1037.0: 30.0, 1039.0: 128.0, 1041.0: 59.0, 1043.0: 211.0, 1045.0: 125.0, 1047.0: 164.0, 1049.0: 197.0, 1051.0: 225.0, 1053.0: 84.0, 1055.0: 375.0, 1057.0: 65.0, 1059.0: 83.0, 1061.0: 86.0, 1063.0: 39.0, 1065.0: 79.0, 1067.0: 45.0, 1069.0: 299.0, 1071.0: 121.0, 1073.0: 1609.0, 1075.0: 38.0, 1077.0: 256.0, 1079.0: 100.0, 1081.0: 185.0, 1083.0: 166.0, 1085.0: 55.0, 1087.0: 54.0, 1089.0: 538.0, 1091.0: 67.0, 1093.0: 108.0, 1095.0: 239.0, 1097.0: 914.0, 1099.0: 42.0, 1101.0: 629.0, 1103.0: 291.0, 1105.0: 28.0, 1107.0: 63.0, 1109.0: 80.0, 1111.0: 53.0, 1113.0: 51.0, 1117.0: 267.0, 1115.0: 252.0, 1119.0: 34.0, 1121.0: 190.0, 1123.0: 157.0, 1125.0: 472.0, 1127.0: 291.0, 1129.0: 39.0, 1131.0: 31.0, 1133.0: 73.0, 2013.0: 3.0, 2016.0: 0.0, 

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1566752,2021-07-30,New Haven,Connecticut,9009.0,93680,2140.0,2021-07-30,-14 days,NaN
1569999,2021-07-31,New Haven,Connecticut,9009.0,93680,2140.0,2021-07-31,-13 days,0.0
1573246,2021-08-01,New Haven,Connecticut,9009.0,93680,2140.0,2021-08-01,-12 days,0.0
1576493,2021-08-02,New Haven,Connecticut,9009.0,93986,2140.0,2021-08-02,-11 days,306.0
1579740,2021-08-03,New Haven,Connecticut,9009.0,94202,2140.0,2021-08-03,-10 days,216.0
1582989,2021-08-04,New Haven,Connecticut,9009.0,94378,2140.0,2021-08-04,-9 days,176.0
1586238,2021-08-05,New Haven,Connecticut,9009.0,94526,2144.0,2021-08-05,-8 days,148.0
1589486,2021-08-06,New Haven,Connecticut,9009.0,94736,2144.0,2021-08-06,-7 days,210.0
1592734,2021-08-07,New Haven,Connecticut,9009.0,94736,2144.0,2021-08-07,-6 days,0.0
1595982,2021-08-08,New Haven,Connecticut,9009.0,94736,2144.0,2021-08-08,-5 days,0.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,2020.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
479,06/22/2021,Richmond,New York,36085.0,17,0.0,2021-06-22
480,06/23/2021,Richmond,New York,36085.0,23,0.0,2021-06-23
481,06/24/2021,Richmond,New York,36085.0,29,0.0,2021-06-24
482,06/25/2021,Richmond,New York,36085.0,23,1.0,2021-06-25
483,06/26/2021,Richmond,New York,36085.0,24,0.0,2021-06-26
484,06/27/2021,Richmond,New York,36085.0,22,0.0,2021-06-27
485,06/28/2021,Richmond,New York,36085.0,30,2.0,2021-06-28
486,06/29/2021,Richmond,New York,36085.0,34,0.0,2021-06-29
487,06/30/2021,Richmond,New York,36085.0,27,0.0,2021-06-30
488,07/01/2021,Richmond,New York,36085.0,33,0.0,2021-07-01


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,53.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-08-14 02:10:37.712085
8


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 114.0
1003.0 344.0
1005.0 65.0
1007.0 67.0
1009.0 140.0
1011.0 41.0
1013.0 72.0
1015.0 339.0
1017.0 125.0
1019.0 49.0
1021.0 117.0
1023.0 25.0
1025.0 63.0
1027.0 59.0
1029.0 45.0
1031.0 137.0
1033.0 143.0
1035.0 33.0
1037.0 30.0
1039.0 128.0
1041.0 59.0
1043.0 211.0
1045.0 125.0
1047.0 164.0
1049.0 197.0
1051.0 225.0
1053.0 84.0
1055.0 375.0
1057.0 65.0
1059.0 83.0
1061.0 86.0
1063.0 39.0
1065.0 79.0
1067.0 45.0
1069.0 299.0
1071.0 121.0
1073.0 1609.0
1075.0 38.0
1077.0 256.0
1079.0 100.0
1081.0 185.0
1083.0 166.0
1085.0 55.0
1087.0 54.0
1089.0 538.0
1091.0 67.0
1093.0 108.0
1095.0 239.0
1097.0 914.0
1099.0 42.0
1101.0 629.0
1103.0 291.0
1105.0 28.0
1107.0 63.0
1109.0 80.0
1111.0 53.0
1113.0 51.0
1117.0 267.0
1115.0 252.0
1119.0 34.0
1121.0 190.0
1123.0 157.0
1125.0 472.0
1127.0 291.0
1129.0 39.0
1131.0 31.0
1133.0 73.0
2013.0 3.0
2016.0 0.0
2020.0 191.0
2050.0 22.0
2997.0 0.0
2068.0 0.0
2070.0 2.0
2090.0 45.0
2100.0 0.0
2110.0 3.0
2122.0 20.0
2130.0 2.0
2150.0 6.0
2158.0 4.0
21

26159.0 138.0
26161.0 315.0
26163.0 5231.0
26165.0 48.0
27001.0 37.0
27003.0 468.0
27005.0 58.0
27007.0 65.0
27009.0 98.0
27011.0 4.0
27013.0 44.0
27015.0 40.0
27017.0 58.0
27019.0 49.0
27021.0 33.0
27023.0 39.0
27025.0 55.0
27027.0 92.0
27029.0 17.0
27031.0 0.0
27033.0 24.0
27035.0 99.0
27037.0 482.0
27039.0 3.0
27041.0 81.0
27043.0 22.0
27045.0 10.0
27047.0 37.0
27049.0 75.0
27051.0 8.0
27053.0 1814.0
27055.0 16.0
27057.0 41.0
27059.0 68.0
27061.0 69.0
27063.0 12.0
27065.0 29.0
27067.0 86.0
27069.0 22.0
27071.0 19.0
27073.0 22.0
27075.0 21.0
27077.0 4.0
27079.0 28.0
27081.0 3.0
27083.0 54.0
27087.0 9.0
27089.0 18.0
27091.0 33.0
27085.0 61.0
27093.0 45.0
27095.0 58.0
27097.0 62.0
27099.0 34.0
27101.0 10.0
27103.0 45.0
27105.0 50.0
27107.0 9.0
27109.0 105.0
27111.0 88.0
27113.0 20.0
27115.0 23.0
27117.0 26.0
27119.0 74.0
27121.0 8.0
27123.0 917.0
27125.0 7.0
27127.0 41.0
27129.0 46.0
27131.0 113.0
27133.0 19.0
27135.0 21.0
27139.0 139.0
27141.0 96.0
27143.0 10.0
27137.0 323.0
27145.0 2

48059.0 41.0
48061.0 1731.0
48063.0 45.0
48065.0 14.0
48067.0 87.0
48069.0 32.0
48071.0 62.0
48073.0 150.0
48075.0 17.0
48077.0 17.0
48079.0 15.0
48081.0 13.0
48083.0 36.0
48085.0 871.0
48087.0 9.0
48089.0 33.0
48091.0 350.0
48093.0 52.0
48095.0 9.0
48097.0 72.0
48099.0 95.0
48101.0 7.0
48103.0 14.0
48105.0 15.0
48107.0 29.0
48109.0 11.0
48111.0 26.0
48113.0 4232.0
48115.0 70.0
48123.0 63.0
48117.0 68.0
48119.0 14.0
48121.0 800.0
48125.0 10.0
48127.0 39.0
48129.0 15.0
48131.0 42.0
48133.0 51.0
48135.0 392.0
48137.0 6.0
48141.0 2765.0
48139.0 329.0
48143.0 71.0
48145.0 33.0
48147.0 112.0
48149.0 71.0
48151.0 14.0
48153.0 30.0
48155.0 10.0
48157.0 760.0
48159.0 26.0
48161.0 54.0
48163.0 57.0
48165.0 46.0
48167.0 474.0
48169.0 22.0
48171.0 54.0
48173.0 2.0
48175.0 16.0
48177.0 60.0
48179.0 58.0
48181.0 353.0
48183.0 369.0
48185.0 74.0
48187.0 233.0
48189.0 159.0
48191.0 14.0
48193.0 29.0
48195.0 24.0
48197.0 12.0
48199.0 122.0
48201.0 6829.0
48203.0 115.0
48205.0 2.0
48207.0 28.0
48209.0 

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,5089,31


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 5089}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 87.0
1001 438.0
1009 460.0
1013 177.0
1015 881.0
1017 310.0
1031 913.0
1033 466.0
1039 485.0
1043 701.0
1045 914.0
1051 696.0
1055 930.0
1067 283.0
1069 1343.0
1071 307.0
1077 716.0
1079 145.0
1083 752.0
1089 2152.0
1095 952.0
1097 8341.0
1103 709.0
1111 162.0
1113 341.0
1115 821.0
1117 1915.0
1121 551.0
2261 73.0
4021 2642.0
5009 438.0
5011 77.0
5033 506.0
5037 161.0
5045 1271.0
5047 207.0
5051 1281.0
5053 286.0
5063 406.0
5083 180.0
5085 809.0
5087 186.0
5115 757.0
5117 90.0
5121 114.0
5125 1206.0
5131 1116.0
5145 1166.0
6007 819.0
6017 927.0
6023 883.0
6027 59.0
6061 1869.0
6079 1209.0
6089 757.0
6093 189.0
8019 22.0
8039 39.0
8047 10.0
8051 28.0
8057 3.0
8065 22.0
8067 158.0
8071 69.0
8075 29.0
8077 328.0
8083 47.0
8085 118.0
8093 58.0
8099 23.0
8101 310.0
8103 16.0
8119 27.0
8121 3.0
9005 285.0
9007 329.0
9011 586.0
9015 178.0
10001 644.0
12019 3004.0
12033 4171.0
12089 1401.0
12101 7278.0
12109 3279.0
12113 2450.0
13015 611.0
13039 545.0
13045 747.0
13047 320.0
13057 1639.0

36053 88.0
36063 286.0
36069 123.0
36073 59.0
36075 229.0
36079 211.0
36083 333.0
36091 583.0
36095 67.0
36097 6.0
36101 68.0
36103 4357.0
36107 87.0
36111 348.0
36113 164.0
36115 93.0
36117 149.0
36121 26.0
37001 789.0
37009 117.0
37021 1129.0
37023 597.0
37025 1497.0
37027 413.0
37031 405.0
37035 1038.0
37037 371.0
37049 428.0
37053 106.0
37055 246.0
37057 974.0
37059 266.0
37069 319.0
37071 1500.0
37077 196.0
37087 339.0
37097 1191.0
37101 1391.0
37109 610.0
37115 98.0
37121 60.0
37123 128.0
37125 667.0
37127 349.0
37139 134.0
37141 405.0
37145 192.0
37151 695.0
37157 349.0
37159 1089.0
37161 441.0
37167 432.0
37169 208.0
37171 421.0
37197 195.0
38001 3.0
38003 14.0
38007 1.0
38009 15.0
38011 6.0
38021 13.0
38025 14.0
38027 0.0
38031 0.0
38033 3.0
38041 0.0
38043 4.0
38049 6.0
38055 20.0
38057 15.0
38059 81.0
38065 1.0
38067 13.0
38069 8.0
38073 2.0
38075 2.0
38081 0.0
38091 3.0
38095 2.0
38097 15.0
38099 16.0
39003 219.0
39005 176.0
39007 99.0
39011 113.0
39013 174.0
39015 166.0
39

21113 324.0
21137 233.0
21139 60.0
21155 143.0
21167 189.0
21173 287.0
21177 283.0
21183 202.0
21187 68.0
21197 83.0
21221 89.0
21225 204.0
21233 174.0
22005 2562.0
22011 521.0
22023 58.0
22053 325.0
22057 2373.0
22073 3006.0
22079 2028.0
22081 72.0
22089 994.0
22099 805.0
22111 323.0
22121 406.0
22123 107.0
22125 147.0
23003 106.0
23015 52.0
24003 948.0
24005 1130.0
24011 40.0
24017 460.0
24027 318.0
24041 51.0
24045 371.0
25001 399.0
25005 1576.0
25015 177.0
25021 1106.0
25027 1354.0
26003 5.0
26009 40.0
26011 15.0
26013 9.0
26019 17.0
26025 256.0
26031 29.0
26039 23.0
26045 202.0
26051 24.0
26057 56.0
26065 492.0
26071 6.0
26077 568.0
26081 984.0
26083 0.0
26089 21.0
26101 45.0
26125 2444.0
26131 4.0
26141 16.0
26161 595.0
27001 23.0
27007 133.0
27009 84.0
27011 5.0
27013 129.0
27019 198.0
27021 49.0
27027 72.0
27037 864.0
27039 86.0
27057 50.0
27073 5.0
27075 11.0
27077 5.0
27081 11.0
27083 56.0
27095 39.0
27099 128.0
27101 9.0
27109 398.0
27113 24.0
27117 2.0
27119 56.0
27131 157.

28115 300.0
29119 226.0
29215 184.0
29221 131.0
29510 1078.0
31037 0.0
32009 0.0
32023 172.0
32027 9.0
34001 656.0
35041 112.0
36061 300.0
37005 31.0
37007 125.0
37065 170.0
37093 308.0
37107 370.0
37163 425.0
37181 154.0
37187 34.0
39075 44.0
40091 107.0
41045 137.0
45025 248.0
45027 251.0
45049 157.0
45065 40.0
47029 328.0
47129 118.0
48055 435.0
48105 27.0
48115 29.0
48137 3.0
48165 64.0
48273 363.0
48291 657.0
48313 88.0
48329 696.0
48345 20.0
48409 339.0
48413 18.0
48439 10825.0
48443 1.0
48449 210.0
48457 172.0
48471 471.0
48473 296.0
48495 34.0
49039 129.0
51029 32.0
51081 20.0
51153 1023.0
51590 168.0
51620 39.0
51660 92.0
51683 62.0
51685 3.0
51690 29.0
51700 785.0
51740 342.0
51760 702.0
53025 563.0
54013 16.0
54109 96.0
1065 109.0
2013 17.0
2185 50.0
4009 112.0
4012 53.0
5035 538.0
5095 61.0
5099 53.0
6039 539.0
6071 9781.0
6075 3520.0
6077 3469.0
6099 2838.0
12013 294.0
12077 196.0
12079 286.0
12086 41364.0
12093 461.0
12123 276.0
12125 285.0
13019 291.0
13063 1705.0
13095 

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 1504
36061 2855
36047 5089 5089
36081 3821
36005 2143


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 438.0, 1003.0: 3882.0, 1005.0: 250.0, 1007.0: 222.0, 1009.0: 460.0, 1011.0: 38.0, 1013.0: 177.0, 1015.0: 881.0, 1017.0: 310.0, 1019.0: 145.0, 1021.0: 283.0, 1023.0: 7.0, 1025.0: 227.0, 1027.0: 83.0, 1029.0: 68.0, 1031.0: 913.0, 1033.0: 466.0, 1035.0: 120.0, 1037.0: 68.0, 1039.0: 485.0, 1041.0: 146.0, 1043.0: 701.0, 1045.0: 914.0, 1047.0: 163.0, 1049.0: 562.0, 1051.0: 696.0, 1053.0: 680.0, 1055.0: 930.0, 1057.0: 96.0, 1059.0: 335.0, 1061.0: 385.0, 1063.0: 31.0, 1065.0: 109.0, 1067.0: 283.0, 1069.0: 1343.0, 1071.0: 307.0, 1073.0: 6037.0, 1075.0: 75.0, 1077.0: 716.0, 1079.0: 145.0, 1081.0: 1242.0, 1083.0: 752.0, 1085.0: 50.0, 1087.0: 102.0, 1089.0: 2152.0, 1091.0: 125.0, 1093.0: 249.0, 1095.0: 952.0, 1097.0: 8341.0, 1099.0: 278.0, 1101.0: 1563.0, 1103.0: 709.0, 1105.0: 46.0, 1107.0: 166.0, 1109.0: 277.0, 1111.0: 162.0, 1113.0: 341.0, 1115.0: 821.0, 1117.0: 1915.0, 1119.0: 43.0, 1121.0: 551.0, 1123.0: 417.0, 1125.0: 1252.0, 1127.0: 544.0, 1129.0: 234.0, 1131.0: 40.0, 1133.0: 211.0

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 775051
ALABAMA 947688
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 283412
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 81.0 38921 208.11387168880552
1001 114.0 55869 204.04875691349406
1009 140.0 57826 242.10562722650712
1013 72.0 19448 370.2180172768408
1015 339.0 113605 298.4023590510981
1017 125.0 33254 375.8946292175377
1031 137.0 52342 261.74009399717244
1033 143.0 55241 258.86569757969625
1039 128.0 37049 345.48840724446
1043 211.0 83768 251.8861617801547
1045 125.0 49172 254.20971284470838
1051 225.0 81209 277.06288711842285
1055 375.0 102268 366.6836155982321
1067 45.0 17205 261.5518744551003
1069 299.0 105882 282.38983018832283
1071 121.0 51626 234.37802657575642
1077 256.0 92729 276.07328883089434
1079 100.0 32924 303.7298019681691
1083 166.0 98915 167.82085629075468
1089 538.0 372909 144.27112244542235
1095 239.0 96774 246.96716060098785
1097 914.0 413210 221.19503400208126
1103 291.0 119679 243.15042739327703
1111 53.0 22722 233.2541149546695
1113 51.0 57961 87.99020030710305
1115 252.0 89512 281.52649924032534
1117 267.0 217702 122.64471617164747
1121 190.0 79978 237.5653304658781
22

30015 7.0 5635 124.22360248447205
30017 18.0 11402 157.8670408700228
30021 32.0 8613 371.53140601416465
30025 2.0 2846 70.27406886858749
30027 31.0 11050 280.54298642533934
30029 113.0 103806 108.8569061518602
30031 63.0 114434 55.0535679955258
30033 2.0 1258 158.9825119236884
30043 8.0 12221 65.46109156370183
30045 1.0 2007 49.82561036372695
30047 40.0 30458 131.32838663077024
30049 94.0 69432 135.38426085954603
30059 7.0 1862 375.9398496240601
30063 103.0 119600 86.12040133779264
30067 21.0 16606 126.4603155485969
30069 1.0 487 205.3388090349076
30071 16.0 3954 404.6535154274153
30073 3.0 5911 50.75283369988157
30075 8.0 1682 475.62425683709864
30081 51.0 43806 116.42240788933022
30083 18.0 10803 166.62038322688142
30091 5.0 3309 151.1030522816561
30095 16.0 9642 165.94067620825555
30099 6.0 6147 97.60858955588093
30103 1.0 696 143.67816091954023
30105 18.0 7396 243.37479718766903
30107 9.0 2126 423.33019755409214
30111 285.0 161300 176.68939863608185
31001 54.0 31363 172.17740649810

42127 84.0 51361 163.5482175191293
42129 785.0 348899 224.99347948833332
42131 54.0 26794 201.53765768455622
42133 840.0 449058 187.05824191975202
44001 146.0 48479 301.16132758514
44005 43.0 82082 52.38663775249141
44009 172.0 125577 136.9677568344522
45003 232.0 170872 135.77414672971582
45007 541.0 202558 267.08399569506014
45015 212.0 227907 93.0203986713879
45023 80.0 32244 248.10817516437166
45035 236.0 162809 144.9551314730758
45045 1049.0 523542 200.36596872839237
45055 145.0 66551 217.8780183618578
45057 176.0 98012 179.5698485899686
45063 489.0 298750 163.68200836820083
45073 146.0 79546 183.5415985718955
45077 314.0 126884 247.47013019766084
45083 836.0 319785 261.4256453554732
45091 401.0 280979 142.71529189014126
46015 10.0 5297 188.78610534264678
46019 22.0 10429 210.95023492185254
46025 5.0 3736 133.83297644539613
46029 82.0 28009 292.76304045128353
46039 9.0 4351 206.84900022983223
46043 9.0 2921 308.1136597055803
46045 14.0 3829 365.6307129798903
46051 42.0 7052 595.57

6057 77.0 99755 77.18911332765275
6063 6.0 18807 31.9030148349019
6091 0.0 3005 0.0
6105 3.0 12285 24.42002442002442
6109 75.0 54478 137.67025221190207
8003 39.0 16233 240.25133986324155
8005 846.0 656590 128.84753042233356
8007 2.0 14029 14.256183619645022
8009 4.0 3581 111.7006422786931
8013 263.0 326196 80.62637187457848
8014 77.0 70465 109.27410771304903
8015 25.0 20356 122.81391235999213
8017 6.0 1831 327.6897870016384
8027 5.0 5068 98.65824782951856
8029 75.0 31162 240.6777485398883
8035 276.0 351154 78.59799404249988
8041 920.0 720403 127.70629772502335
8043 66.0 47839 137.96275005748447
8049 7.0 15734 44.489640269480105
8059 819.0 582881 140.50895465798337
8061 3.0 1406 213.37126600284498
8063 13.0 7097 183.17598985486825
8069 250.0 356899 70.04782865740728
8079 1.0 769 130.03901170351105
8081 29.0 13283 218.32417375592865
8091 5.0 4952 100.96930533117933
8095 13.0 4265 304.8065650644783
8097 4.0 17767 22.51364889964541
8107 21.0 25638 81.90966534051017
8113 1.0 8179 12.2264335

26081 874.0 656955 133.0380315242292
26083 2.0 2116 94.5179584120983
26089 20.0 21761 91.90754101374017
26101 26.0 24558 105.87181366560795
26125 2485.0 1257584 197.60111451799642
26131 22.0 5720 384.61538461538464
26141 34.0 12592 270.012706480305
26161 315.0 367601 85.6907353353228
27001 37.0 15886 232.9094800453229
27007 65.0 47188 137.74688480122063
27009 98.0 40889 239.67326175744086
27011 4.0 4991 80.14425966740133
27013 44.0 67653 65.03776624835558
27019 49.0 105089 46.6271446107585
27021 33.0 29779 110.81634709023137
27027 92.0 64222 143.2530908411448
27037 482.0 429021 112.34881276207923
27039 3.0 20934 14.330753797649757
27057 41.0 21491 190.77753478200177
27073 22.0 6623 332.1757511701646
27075 21.0 10641 197.34987313222442
27077 4.0 3740 106.95187165775401
27081 3.0 5639 53.20092214931725
27083 54.0 25474 211.98084321268746
27095 58.0 26277 220.7253491646687
27099 34.0 40062 84.8684538964605
27101 10.0 8194 122.040517451794
27109 105.0 158293 66.33268685286147
27113 20.0 14

8073 4.0 5701 70.1631292755657
8087 96.0 29068 330.2600798128526
8109 8.0 6824 117.23329425556857
8111 0.0 728 0.0
8117 8.0 31011 25.79729773306246
8125 17.0 10019 169.67761253618127
9001 2204.0 943332 233.63990620481442
9009 2146.0 854757 251.06550750681188
12003 62.0 29210 212.25607668606642
12007 58.0 28201 205.6664657281657
12023 170.0 71686 237.1453282370337
12055 366.0 106221 344.5646341118988
12057 1832.0 1471968 124.45922737450813
12061 306.0 159923 191.34208337762547
12065 28.0 14246 196.54639898918995
12081 689.0 403253 170.86047716941968
12085 335.0 161000 208.07453416149067
12087 52.0 74228 70.05442690089993
12099 2883.0 1496770 192.61476379136408
12115 844.0 433742 194.58572146575614
12129 56.0 33739 165.98002311864607
13007 11.0 3038 362.08031599736665
13011 44.0 19234 228.7615680565665
13023 52.0 12873 403.9462440767498
13029 42.0 39627 105.98834128245893
13031 125.0 79608 157.01939503567482
13053 17.0 10907 155.863207114697
13075 44.0 17270 254.77707006369425
13079 21.0

54055 129.0 58758 219.5445726539365
54075 12.0 8247 145.50745725718443
54089 22.0 12573 174.97812773403325
54099 46.0 39402 116.74534287599614
54105 4.0 5821 68.7167153410067
56039 11.0 23464 46.88032730992158
1025 63.0 23622 266.7005334010668
1029 45.0 14910 301.8108651911469
1059 83.0 31362 264.65148906319746
1133 73.0 23629 308.9424012865547
2060 0 836 0.0
2105 0 2148 0.0
2150 6.0 12998 46.16094783812894
2170 62.0 108317 57.239399170951934
2198 4.0 6203 64.48492664839593
2240 6.0 6893 87.04482808646452
4007 239.0 54018 442.4451108889629
4013 10575.0 4485414 235.76419032891948
4015 813.0 212181 383.16343122145713
5049 51.0 12477 408.75210387112287
5057 20.0 21532 92.88500835965075
5065 47.0 13629 344.8528872257686
5067 38.0 16719 227.28632095221005
5089 39.0 16694 233.61686833592907
5097 33.0 8986 367.2379256621411
5101 27.0 7753 348.2522894363472
5111 87.0 23528 369.77218633117985
5127 18.0 10281 175.08024511234316
5137 35.0 12506 279.8656644810491
5147 16.0 6320 253.16455696202533


48113 4232.0 2635516 160.57576580828953
48117 68.0 18546 366.65588267011753
48153 30.0 5712 525.2100840336135
48169 22.0 6229 353.1867073366511
48177 60.0 20837 287.94932091951813
48195 24.0 5399 444.52676421559545
48201 6829.0 4713325 144.88710199275457
48207 28.0 5658 494.87451396253095
48249 112.0 40482 276.66617261992985
48255 37.0 15601 237.16428434074737
48271 7.0 3667 190.89173711480777
48279 83.0 12893 643.7601799426045
48301 0.0 169 0.0
48341 75.0 20940 358.1661891117479
48357 29.0 9836 294.83529890199264
48369 35.0 9605 364.3935450286309
48371 35.0 15823 221.19699172091256
48375 472.0 117415 401.99293105650895
48389 45.0 15976 281.6725087631447
48419 74.0 25274 292.79101052464983
48435 8.0 3776 211.864406779661
48437 18.0 7397 243.341895362985
48463 76.0 26741 284.20777083878687
48465 233.0 49025 475.26772055073945
48501 29.0 8713 332.8359921955698
51027 47.0 21004 223.76690154256332
51051 19.0 14318 132.70009777901942
51105 48.0 23423 204.92678136874014
51183 23.0 11159 206.

72051 nan 36141 nan
72053 nan 29454 nan
72057 nan 39465 nan
72067 nan 15518 nan
72117 nan 13656 nan
72125 nan 30227 nan
72133 nan 21209 nan
72143 nan 36061 nan
72015 nan 17238 nan
72045 nan 18648 nan
72047 nan 32293 nan
72049 nan 1714 nan
72081 nan 24276 nan
72087 nan 24553 nan
72095 nan 10321 nan
72097 nan 71530 nan
72107 nan 20220 nan
72113 nan 131881 nan
72123 nan 27128 nan
72127 nan 318441 nan
72083 nan 7927 nan
72001 nan 17363 nan
72055 nan 15383 nan
72073 nan 13891 nan
72079 nan 22010 nan
72093 nan 5430 nan
72019 nan 27725 nan


In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(2146.0, 854757, 251.06550750681188)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 7861.897052732757,
 'ALABAMA': 19336.236944636203,
 'ALASKA': 931.4389796341509,
 'ARIZONA': 4834.083417940147,
 'ARKANSAS': 18641.53981149919,
 'CALIFORNIA': 6457.0605655537665,
 'COLORADO': 8738.692106324157,
 'CONNECTICUT': 1618.230317655952,
 'DELAWARE': 607.7145678741354,
 'FLORIDA': 13229.584821335615,
 'GEORGIA': 46037.36460877037,
 'IDAHO': 5531.579167036993,
 'ILLINOIS': 24362.66951520937,
 'INDIANA': 20542.013111115342,
 'IOWA': 25522.14844405859,
 'KANSAS': 28617.86089909923,
 'KENTUCKY': 23295.391145520087,
 'LOUISIANA': 18637.90330229182,
 'MAINE': 958.1008381438053,
 'MARYLAND': 3941.3534447056854,
 'MASSACHUSETTS': 3042.0318321979667,
 'MICHIGAN': 16490.39649481993,
 'MINNESOTA': 13361.199463454905,
 'MISSISSIPPI': 25724.83622033877,
 'MISSOURI': 19693.846099079827,
 'MONTANA': 12302.218866469711,
 'NEBRASKA': 14849.396907654907,
 'NEVADA': 2286.6305777297375,
 'NEW HAMPSHIRE': 875.0335728226802,
 'NEW JERSEY': 5855.570343946895,
 'NEW YORK': 9409.62282808

In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 775051.0, 'ALABAMA': 947688.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 283412.0, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON': 1053931

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 775051.0 76321890.0 1000000 10155.028917653899
ALABAMA 947688.0000000001 76321890.0 1000000 12416.988101316674
ALASKA 231145.0 76321890.0 1000000 3028.554455346952
ARIZONA 1839932.0 76321890.0 1000000 24107.526687297704
ARKANSAS 830467.0000000003 76321890.0 1000000 10881.111565764428
CALIFORNIA 11847711.0 76321890.0 1000000 155233.45923430356
COLORADO 1337805.0 76321890.0 1000000 17528.45743206831
CONNECTICUT 874974.0 76321890.0 1000000 11464.260122489104
DELAWARE 239009.0 76321890.0 1000000 3131.591736001297
FLORIDA 3892551.999999999 76321890.0 1000000 51001.77681658564
GEORGIA 1928702.999999999 76321890.0 1000000 25270.639917329077
IDAHO 340742.0000000001 76321890.0 1000000 4464.538286460151
ILLINOIS 2987496.0000000005 76321890.0 1000000 39143.37027031171
INDIANA 1602975.9999999995 76321890.0 1000000 21002.834180338035
IOWA 699741.0 76321890.0 1000000 9168.287106097609
KANSAS 401103.0000000001 76321890.0 1000000 5255.41230700655
KENTUCKY 1416013.0000000002 76321890.0 10000

In [98]:
print (Covid_state)

{'NEW MEXICO': 8153.0, 'ALABAMA': 47712.0, 'ALASKA': 4247.0, 'ARIZONA': 33823.0, 'ARKANSAS': 30593.0, 'CALIFORNIA': 157773.0, 'COLORADO': 14808.0, 'CONNECTICUT': 7478.0, 'DELAWARE': 2574.0, 'FLORIDA': 286516.0, 'GEORGIA': 70935.0, 'IDAHO': 7573.0, 'ILLINOIS': 38516.0, 'INDIANA': 25442.0, 'IOWA': 8435.0, 'KANSAS': 14968.0, 'KENTUCKY': 29805.0, 'LOUISIANA': 73078.0, 'MAINE': 1970.0, 'MARYLAND': 10108.0, 'MASSACHUSETTS': 13560.0, 'MICHIGAN': 17461.0, 'MINNESOTA': 12149.0, 'MISSISSIPPI': 37642.0, 'MISSOURI': 35156.0, 'MONTANA': 3496.0, 'NEBRASKA': 3428.0, 'NEVADA': 14725.0, 'NEW HAMPSHIRE': 2106.0, 'NEW JERSEY': 18917.0, 'NEW YORK': 40269.0, 'NORTH CAROLINA': 59438.0, 'NORTH DAKOTA': 1495.0, 'OHIO': 27517.0, 'OKLAHOMA': 25395.0, 'OREGON': 18708.0, 'PENNSYLVANIA': 21889.0, 'RHODE ISLAND': 2413.0, 'SOUTH CAROLINA': 39166.0, 'SOUTH DAKOTA': 1537.0, 'TENNESSEE': 45065.0, 'TEXAS': 187386.0, 'UTAH': 11910.0, 'VERMONT': 1152.0, 'VIRGINIA': 23470.0, 'WASHINGTON': 33969.0, 'WEST VIRGINIA': 4997.0, 

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 7861.897052732757,
 'ALABAMA': 19336.236944636203,
 'ALASKA': 931.4389796341509,
 'ARIZONA': 4834.083417940147,
 'ARKANSAS': 18641.53981149919,
 'CALIFORNIA': 6457.0605655537665,
 'COLORADO': 8738.692106324157,
 'CONNECTICUT': 1618.230317655952,
 'DELAWARE': 607.7145678741354,
 'FLORIDA': 13229.584821335615,
 'GEORGIA': 46037.36460877037,
 'IDAHO': 5531.579167036993,
 'ILLINOIS': 24362.66951520937,
 'INDIANA': 20542.013111115342,
 'IOWA': 25522.14844405859,
 'KANSAS': 28617.86089909923,
 'KENTUCKY': 23295.391145520087,
 'LOUISIANA': 18637.90330229182,
 'MAINE': 958.1008381438053,
 'MARYLAND': 3941.3534447056854,
 'MASSACHUSETTS': 3042.0318321979667,
 'MICHIGAN': 16490.39649481993,
 'MINNESOTA': 13361.199463454905,
 'MISSISSIPPI': 25724.83622033877,
 'MISSOURI': 19693.846099079827,
 'MONTANA': 12302.218866469711,
 'NEBRASKA': 14849.396907654907,
 'NEVADA': 2286.6305777297375,
 'NEW HAMPSHIRE': 875.0335728226802,
 'NEW JERSEY': 5855.570343946895,
 'NEW YORK': 9409.62282808

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)